In [1]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime

In [2]:
base_path = '/Users/yifanli/Github/fidelity-portfolio-tracker'
os.chdir(base_path)

In [8]:
from support_functions import portfolio, data_loader

In [9]:
importlib.reload(data_loader)

<module 'support_functions.data_loader' from '/Users/yifanli/Github/fidelity-portfolio-tracker/code/support_functions/data_loader.py'>

In [109]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'

transactions = data_loader.load_transaction(data_folder_path, transaction_file_pattern)
position = data_loader.load_position(data_folder_path, position_file_pattern)

# Attribute 1: get investment distribution

In [129]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [130]:
current_portfolio.get_investment_distribution()

{'Class': ['stock', 'bill', 'other', 'total'],
 'Amount': [240727.72, 316079.98, 43192.30000000005, 600000.0],
 'Percent': [0.40121286666666667,
  0.5267999666666666,
  0.07198716666666674,
  1.0]}

In [131]:
current_portfolio.show_investment_distribution()

   Class     Amount  Percent
0  stock  240727.72   40.12%
1   bill  316079.98   52.68%
2  other   43192.30    7.20%
3  total  600000.00  100.00%


# Attribute 2: get overall IRR

In [26]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [27]:
current_portfolio.get_overall_irr()

0.12285282721063749

# Attribute 3: get IRR for each stock

In [127]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [133]:
current_portfolio.get_stock_irr()

Total current value has been added


{'Transfer': 0.12285282721063749,
 'AAPL': 0.19783757778971012,
 'SBUX': 0.1507940862201078,
 'JPM': 0.395893293677582,
 'AXP': 0.3713396509117076,
 'GOOGL': 0.48413322302850126,
 'AMZN': 0.5689077515810341,
 'NKE': -0.1063227652772601,
 'TSLA': 1.1818276805875267,
 'FXAIX': 0.3215858142325763,
 'MCD': 0.3987630224319425,
 'FSKAX': 0.3701158034150592,
 'FSPSX': 0.0105729623745894,
 'COKE': 0.39188667056067217,
 'MSFT': -0.16677724293162943,
 'BRKB': 0.3015211187482966}

In [115]:
current_portfolio.add_total_current_value_to_individual_position()

In [116]:
current_portfolio.set_merged_individual_position_transaction()

In [119]:
current_portfolio.merged_individual_position_transaction['Symbol'].unique()

array(['Transfer', 'FZFXX', 'AAPL', 'SBUX', 'JPM', 'AXP', 'GOOGL', 'AMZN',
       'NKE', '912797JK4', '912797LL9', '912797GN1', '912797GZ4',
       '912797HG5', 'TSLA', 'FXAIX', '912797HQ3', '912797HT7', 'MCD',
       '912797GB7', '912797HS9', '912797JT5', '912797KV8', '912797JU2',
       '912797KB2', '912797KC0', 'FSKAX', 'FSPSX', 'COKE', 'MSFT',
       '912797KL0', '912797LH8', '912797KA4', '912797LE5', '912797GW1',
       '912797HP5', '912797KT3', '912797LC9', '912797LR6', '912797LX3',
       '912797MB0', '912797LY1', 'BRKB', '912797LZ8', '912797LP0',
       '912796ZV4', 'FZFXX**'], dtype=object)

In [122]:
irr_dict = {}
for symbol in current_portfolio.merged_individual_position_transaction['Symbol'].unique():
    if not symbol[0].isdigit() and symbol != 'FZFXX**' and symbol != 'FZFXX':
        irr_dict[symbol] = current_portfolio.calculate_irr(symbol)
irr_dict

/Users/yifanli/Github/fidelity-portfolio-tracker/code/support_functions/portfolio.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions["time_diffs_in_year"] = (
/Users/yifanli/Github/fidelity-portfolio-tracker/code/support_functions/portfolio.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions["time_diffs_in_year"] = (
/Users/yifanli/Github/fidelity-portfolio-tracker/code/support_functions/portfolio.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

{'Transfer': 0.12285282721063749,
 'AAPL': 0.19783757778971012,
 'SBUX': 0.1507940862201078,
 'JPM': 0.395893293677582,
 'AXP': 0.3713396509117076,
 'GOOGL': 0.48413322302850126,
 'AMZN': 0.5689077515810341,
 'NKE': -0.1063227652772601,
 'TSLA': 1.1818276805875267,
 'FXAIX': 0.3215858142325763,
 'MCD': 0.3987630224319425,
 'FSKAX': 0.3701158034150592,
 'FSPSX': 0.0105729623745894,
 'COKE': 0.39188667056067217,
 'MSFT': -0.16677724293162943,
 'BRKB': 0.3015211187482966}

In [108]:
transactions = current_portfolio.filter_merged_transactions_by_symbol(symbol)
transactions["time_diffs_in_year"] = (
    current_portfolio.today - transactions["Run Date"]
).apply(lambda x: x.days) / 365.25
transactions

/var/folders/r_/js4cr3ps49j4qkf0dy2w4lm00000gn/T/ipykernel_67314/711882221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions["time_diffs_in_year"] = (


,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date,time_diffs_in_year
213,2024-11-13,NaN,NaN,BRKB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4673.6,NaN,0.0


In [99]:
current_portfolio.calculate_irr('AAPL')

/Users/yifanli/Github/fidelity-portfolio-tracker/code/support_functions/portfolio.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions["time_diffs_in_year"] = (


0.19783757778971012

In [221]:
current_portfolio.individual_transactions['Symbol'].unique()

array(['Transfer', 'FZFXX', 'AAPL', 'SBUX', 'JPM', 'AXP', 'GOOGL', 'AMZN',
       'NKE', '912797JK4', '912797LL9', '912797GN1', '912797GZ4',
       '912797HG5', 'TSLA', 'FXAIX', '912797HQ3', '912797HT7', 'MCD',
       '912797GB7', '912797HS9', '912797JT5', '912797KV8', '912797JU2',
       '912797KB2', '912797KC0', 'FSKAX', 'FSPSX', 'COKE', 'MSFT',
       '912797KL0', '912797LH8', '912797KA4', '912797LE5', '912797GW1',
       '912797HP5', '912797KT3', '912797LC9', '912797LR6', '912797MB0'],
      dtype=object)

In [ ]:
new_rows = pd.DataFrame({
    "Run Date": [current_portfolio.today] * len(current_portfolio.individual_position),
    "Symbol": current_portfolio.individual_position["Symbol"],
    "Amount ($)": current_portfolio.individual_position["Current Value"]
})

individual_transactions = pd.concat([current_portfolio.individual_transactions, new_rows], ignore_index=True)

In [229]:
irr_dict = {}
for symbol in individual_transactions['Symbol'].unique():
    if not symbol == 'Transfer':
        sub_transactions = individual_transactions[individual_transactions['Symbol']==symbol]
        sub_transactions['time_diffs_in_year'] = (current_portfolio.today - sub_transactions['Run Date']).apply(lambda x: x.days)/365.25
        irr_dict[symbol] = calculate_irr(sub_transactions)

/var/folders/r_/js4cr3ps49j4qkf0dy2w4lm00000gn/T/ipykernel_35122/3531177934.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_transactions['time_diffs_in_year'] = (current_portfolio.today - sub_transactions['Run Date']).apply(lambda x: x.days)/365.25
/var/folders/r_/js4cr3ps49j4qkf0dy2w4lm00000gn/T/ipykernel_35122/3531177934.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_transactions['time_diffs_in_year'] = (current_portfolio.today - sub_transactions['Run Date']).apply(lambda x: x.days)/365.2

ValueError: f(a) and f(b) must have different signs

In [231]:
sub_transactions

,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date,time_diffs_in_year
114,2024-04-22,Individual Z23390746,YOU BOUGHT UNITED STATES TREAS BILLS ZERO CPN...,912797KV8,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 1...,Cash,60000.0,97.39,NaN,NaN,NaN,-58434.8,NaT,0.561259
186,2024-09-10,Individual Z23390746,YOU BOUGHT UNITED STATES TREAS BILLS ZERO CPN...,912797KV8,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 1...,Cash,10000.0,99.42,NaN,NaN,NaN,-9941.9,NaT,0.175222


In [217]:
current_portfolio.individual_transactions[current_portfolio.individual_transactions['Symbol']=='912797HS9']

,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
163,2024-04-16,Individual Z23390746,YOU BOUGHT UNITED STATES TREAS BILLS ZERO CPN...,912797HS9,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 0...,Cash,30000.0,99.38,NaN,NaN,NaN,-29815.03,NaT
196,2024-05-30,Individual Z23390746,REDEMPTION PAYOUT UNITED STATES TREAS BILLS Z...,912797HS9,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 0...,Cash,-30000.0,1.00,NaN,NaN,NaN,30000.00,NaT


In [28]:
current_portfolio.individual_transactions

,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
0,2022-07-26,Individual Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.00,NaN,NaN,NaN,NaN,1000.00,NaT
1,2022-07-29,Individual Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,0.22,NaT
2,2022-07-29,Individual Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.22,1.00,NaN,NaN,NaN,-0.22,NaT
3,2022-08-05,Individual Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,9.00,164.61,NaN,NaN,NaN,-1481.45,2022-08-09
4,2022-08-05,Individual Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,1.00,163.54,NaN,NaN,NaN,-163.54,2022-08-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,2024-09-30,Individual Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,83.80,1.00,NaN,NaN,NaN,-83.80,NaT
312,2024-11-07,Individual Z23390746,REDEMPTION PAYOUT UNITED STATES TREAS BILLS Z...,912797LC9,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 1...,Cash,-30000.00,1.00,NaN,NaN,NaN,30000.00,NaT
313,2024-11-08,Individual Z23390746,DIVIDEND RECEIVED AMERICAN EXPRESS CO COM USD...,AXP,AMERICAN EXPRESS CO COM USD0.20,Cash,0.00,NaN,NaN,NaN,NaN,7.00,NaT
314,2024-11-08,Individual Z23390746,DIVIDEND RECEIVED COCA COLA CONS INC COM (COK...,COKE,COCA COLA CONS INC COM,Cash,0.00,NaN,NaN,NaN,NaN,5.00,NaT


In [160]:
time_diffs = np.array([(datetime.now().date() - date).days for date in dates]) / 365.25
def npv(rate, cash_flows, time_diffs):
    return sum(cash_flows * (1 + rate) ** time_diffs)
for rate in np.arange(-0.999, 5, 0.001):
        if npv(rate, -cash_flows, time_diffs) > PV:
            break
rate

0.19800000000000106

In [207]:
from scipy.optimize import root_scalar

def calculate_irr(sub_transactions_copy, lower_bound=-0.999, upper_bound=5):
    def npv(rate):
        return np.sum(sub_transactions_copy['Amount ($)'] * (1 + rate) ** sub_transactions_copy['time_diffs_in_year'])
    result = root_scalar(npv, bracket=[lower_bound, upper_bound], method='bisect')
    
    if result.converged:
        return result.root
    else:
        raise ValueError("No solution found for the rate that satisfies the equation.")